In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [25]:
DATASET = DatasetPath('ucidata-zachary')

In [26]:
spark = (SparkSession.builder
         .appName(f'{DATASET}_preprocess')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .getOrCreate())

In [27]:
schema = T.StructType([
    T.StructField('src', T.IntegerType(), False),
    T.StructField('dst', T.IntegerType(), False),
])

df = (
    spark.read.csv(DATASET.raw_str('karate_club.csv'), sep=' ', header=False, schema=schema)
        .withColumnRenamed('_c0', 'src')
        .withColumnRenamed('_c1', 'dst')
)
df.head(5)

[Row(src=0, dst=1),
 Row(src=0, dst=2),
 Row(src=0, dst=3),
 Row(src=0, dst=4),
 Row(src=0, dst=5)]

In [28]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id=31), Row(id=28), Row(id=26), Row(id=27), Row(id=22)]

In [29]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

[Row(src=22, dst=33),
 Row(src=4, dst=10),
 Row(src=5, dst=16),
 Row(src=19, dst=33),
 Row(src=0, dst=13)]

In [30]:
df_nodes.write.parquet(DATASET.processed_str('nodes_Pupil'), mode='overwrite')
df_edges.write.parquet(DATASET.processed_str('nodes_TRAINS'), mode='overwrite')

In [31]:
from datasets.formats import coms_to_comlist, write_comlist
import json

with open(DATASET.raw_str('karate_club.json')) as f:
    meta = json.load(f)

coms = {
    cid: nids
    for cid, nids in enumerate(meta['communities'])
}

comlist = coms_to_comlist(coms)
write_comlist(comlist, DATASET.processed_str('ground_truth.comlist'))

In [32]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('Pupil', DATASET.processed_str('nodes_Pupil')),
    ],
    edges=[
        ('Trains', 'Pupil', 'Pupil', DATASET.processed_str('nodes_TRAINS')),
    ],
    ground_truth=str(DATASET.processed_str('ground_truth.ncomlist'))
)

[2022-01-21 23:51:59,814][/dd_volume/Development/Python/Thesis/code/datasets/datasets/build_schema.py][DEBUG] Merging old schema for ucidata-zachary


DatasetSchema(name='ucidata-zachary', prefix='UcidataZachary', database='ucidata-zachary', description="Zachary's karate club dataset", nodes=[NodeSchema(path='data/processed/ucidata-zachary/nodes_Pupil', properties=[Property(name='id', type='int', ignore=False, label=True, timestamp=False)], label='Pupil', interaction=False)], edges=[EdgeSchema(path='data/processed/ucidata-zachary/nodes_TRAINS', properties=[Property(name='src', type='int', ignore=False, label=False, timestamp=False), Property(name='dst', type='int', ignore=False, label=False, timestamp=False)], type='TRAINS', source='Pupil', target='Pupil', directed=False, interaction=False)], ground_truth=PosixPath('data/processed/ucidata-zachary/ground_truth.comlist'), versions={})